In [1]:
#Import des librairies utiles
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

##### 1. Importer les données du fichier quarterly.xls (corriger le problème éventuel d’observations manquantes)

In [2]:
df = pd.read_excel('quarterly.xls')
df.head()

,DATE,FFR,Tbill,Tb1yr,r5,r10,PPINSA,Finished,CPI,CPICORE,M1NSA,M2SA,M2NSA,Unemp,IndProd,RGDP,Potent,Deflator,Curr
0,1960Q1,3.93,3.87,4.57,4.64,4.49,31.67,33.20,29.40,18.92,140.53,896.1,299.40,5.13,23.93,2845.3,2824.2,18.521,31.830
1,1960Q2,3.70,2.99,3.87,4.30,4.26,31.73,33.40,29.57,19.00,138.40,903.3,300.03,5.23,23.41,2832.0,2851.2,18.579,31.862
2,1960Q3,2.94,2.36,3.07,3.67,3.83,31.63,33.43,29.59,19.07,139.60,919.4,305.50,5.53,23.02,2836.6,2878.7,18.648,32.217
3,1960Q4,2.30,2.31,2.99,3.75,3.89,31.70,33.67,29.78,19.14,142.67,932.8,312.30,6.27,22.47,2800.2,2906.7,18.700,32.624
4,1961Q1,2.00,2.35,2.87,3.64,3.79,31.80,33.63,29.84,19.17,142.23,948.9,317.10,6.80,22.13,2816.9,2934.8,18.743,32.073


In [11]:
#Conversion des données de temps au format correct
df.columns = df.columns.str.strip()
df['Datetime'] = pd.to_datetime(df['DATE'])
df = df.set_index('Datetime')
df.reset_index()
df.head()

,DATE,FFR,Tbill,Tb1yr,r5,r10,PPINSA,Finished,CPI,CPICORE,M1NSA,M2SA,M2NSA,Unemp,IndProd,RGDP,Potent,Deflator,Curr
Datetime,,,,,,,,,,,,,,,,,,,
1960-01-01,1960Q1,3.93,3.87,4.57,4.64,4.49,31.67,33.20,29.40,18.92,140.53,896.1,299.40,5.13,23.93,2845.3,2824.2,18.521,31.830
1960-04-01,1960Q2,3.70,2.99,3.87,4.30,4.26,31.73,33.40,29.57,19.00,138.40,903.3,300.03,5.23,23.41,2832.0,2851.2,18.579,31.862
1960-07-01,1960Q3,2.94,2.36,3.07,3.67,3.83,31.63,33.43,29.59,19.07,139.60,919.4,305.50,5.53,23.02,2836.6,2878.7,18.648,32.217
1960-10-01,1960Q4,2.30,2.31,2.99,3.75,3.89,31.70,33.67,29.78,19.14,142.67,932.8,312.30,6.27,22.47,2800.2,2906.7,18.700,32.624
1961-01-01,1961Q1,2.00,2.35,2.87,3.64,3.79,31.80,33.63,29.84,19.17,142.23,948.9,317.10,6.80,22.13,2816.9,2934.8,18.743,32.073


In [12]:
df = df.drop(['DATE'], axis = 1)
df.head()

,FFR,Tbill,Tb1yr,r5,r10,PPINSA,Finished,CPI,CPICORE,M1NSA,M2SA,M2NSA,Unemp,IndProd,RGDP,Potent,Deflator,Curr
Datetime,,,,,,,,,,,,,,,,,,
1960-01-01,3.93,3.87,4.57,4.64,4.49,31.67,33.20,29.40,18.92,140.53,896.1,299.40,5.13,23.93,2845.3,2824.2,18.521,31.830
1960-04-01,3.70,2.99,3.87,4.30,4.26,31.73,33.40,29.57,19.00,138.40,903.3,300.03,5.23,23.41,2832.0,2851.2,18.579,31.862
1960-07-01,2.94,2.36,3.07,3.67,3.83,31.63,33.43,29.59,19.07,139.60,919.4,305.50,5.53,23.02,2836.6,2878.7,18.648,32.217
1960-10-01,2.30,2.31,2.99,3.75,3.89,31.70,33.67,29.78,19.14,142.67,932.8,312.30,6.27,22.47,2800.2,2906.7,18.700,32.624
1961-01-01,2.00,2.35,2.87,3.64,3.79,31.80,33.63,29.84,19.17,142.23,948.9,317.10,6.80,22.13,2816.9,2934.8,18.743,32.073


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 212 entries, 1960-01-01 to 2012-10-01
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FFR       212 non-null    float64
 1   Tbill     212 non-null    float64
 2   Tb1yr     212 non-null    float64
 3   r5        212 non-null    float64
 4   r10       212 non-null    float64
 5   PPINSA    212 non-null    float64
 6   Finished  212 non-null    float64
 7   CPI       212 non-null    float64
 8   CPICORE   212 non-null    float64
 9   M1NSA     212 non-null    float64
 10  M2SA      212 non-null    float64
 11  M2NSA     212 non-null    float64
 12  Unemp     212 non-null    float64
 13  IndProd   212 non-null    float64
 14  RGDP      212 non-null    float64
 15  Potent    212 non-null    float64
 16  Deflator  212 non-null    float64
 17  Curr      212 non-null    float64
dtypes: float64(18)
memory usage: 31.5 KB


In [17]:
#Vérification de valeurs manquantes
df.isnull().sum()

FFR         0
Tbill       0
Tb1yr       0
r5          0
r10         0
PPINSA      0
Finished    0
CPI         0
CPICORE     0
M1NSA       0
M2SA        0
M2NSA       0
Unemp       0
IndProd     0
RGDP        0
Potent      0
Deflator    0
Curr        0
dtype: int64

In [20]:
#Verification rapide de l'existence ou non de valeurs manquantes
df.isnull().values.any()

False

In [3]:
conda install -c conda-forge altair vega_datasets

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::conda-pack==0.6.0=pyhd3eb1b0_0
  - defaults/noarch::jupyterlab_server==2.8.2=pyhd3eb1b0_0
  - conda-forge/osx-64::conda==4.11.0=py39h6e9494a_0
  - defaults/noarch::cookiecutter==1.7.2=pyhd3eb1b0_0
  - defaults/osx-64::astroid==2.6.6=py39hecd8cb5_0
  - defaults/osx-64::numba==0.54.1=py39hae1ba45_0
  - defaults/osx-64::jupyter==1.0.0=py39hecd8cb5_7
  - defaults/noarch::python-lsp-black==1.0.0=pyhd3eb1b0_0
  - defaults/osx-64::gevent==21.8.0=py39h9ed2024_1
  - defaults/osx-64::zope.event==4.5.0=py39hecd8cb5_0
  - defaults/noarch::nbclient==0.5.3=pyhd3eb1b0_0
  - defaults/noarch::conda-repo-cli==1.0.4=pyhd3eb1b0_0
  - defaults/noarch::backports.functools_lru_cache==1.6.4=pyhd3eb1b0_0
  - defaults/osx-64::nbconvert==6.1.0=py39hecd8cb5_0
  - defaults/osx-64::ipykernel==6.4.1=py39hecd8cb5_1
  - defaults/noarch::nbclassic==

done


==> WARNING: A newer version of conda exists. <==
  current version: 4.11.0
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /usr/local/anaconda3

  added / updated specs:
    - altair
    - vega_datasets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.2.0               |     pyhd8ed1ab_1         711 KB  conda-forge
    ca-certificates-2021.10.8  |       h033912b_0         139 KB  conda-forge
    openssl-1.1.1o             |       hfe4f2af_0         1.9 MB  conda-forge
    setuptools-62.2.0          |   py39h6e9494a_0         1.3 MB  conda-forge
    statsmodels-0.13.2         |   py39hc89836e_0        10.7 MB  conda-forge
    vega_datasets-0.9.0        |     pyhd3deb0d_0         179 KB  conda-forge
    ------------------------------------------------------------
                    

In [2]:
conda update -n base conda

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::conda-pack==0.6.0=pyhd3eb1b0_0
  - defaults/noarch::jupyterlab_server==2.8.2=pyhd3eb1b0_0
  - conda-forge/osx-64::conda==4.11.0=py39h6e9494a_0
  - defaults/noarch::cookiecutter==1.7.2=pyhd3eb1b0_0
  - defaults/osx-64::astroid==2.6.6=py39hecd8cb5_0
  - defaults/osx-64::numba==0.54.1=py39hae1ba45_0
  - defaults/osx-64::jupyter==1.0.0=py39hecd8cb5_7
  - defaults/noarch::python-lsp-black==1.0.0=pyhd3eb1b0_0
  - defaults/osx-64::gevent==21.8.0=py39h9ed2024_1
  - defaults/osx-64::zope.event==4.5.0=py39hecd8cb5_0
  - defaults/noarch::nbclient==0.5.3=pyhd3eb1b0_0
  - defaults/noarch::conda-repo-cli==1.0.4=pyhd3eb1b0_0
  - defaults/noarch::backports.functools_lru_cache==1.6.4=pyhd3eb1b0_0
  - defaults/osx-64::nbconvert==6.1.0=py39hecd8cb5_0
  - defaults/osx-64::ipykernel==6.4.1=py39hecd8cb5_1
  - defaults/noarch::nbclassic==

done

## Package Plan ##

  environment location: /usr/local/anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB
    c-ares-1.18.1              |       hca72f7f_0         100 KB
    ca-certificates-2022.4.26  |       hecd8cb5_0         124 KB
    curl-7.82.0                |       hca72f7f_0          84 KB
    freetype-2.11.0            |       hd8bbffd_0         589 KB
    jpeg-9e                    |       hca72f7f_0         225 KB
    libcurl-7.82.0             |       h6dfd666_0         314 KB
    libiconv-1.16              |       hca72f7f_2         695 KB
    libllvm11-11.1.0           |       h46f1229_1        17.7 MB
    libnghttp2-1.46.0          |       ha29bfda_0         687 KB
    libssh2-1.10.0             |       h0a4fc7d_0         258 KB
    libwebp-1.2.2             

##### 2. Stationnariser la série de CPI en utilisant la méthode de régression qui inclue un terme de tendance dont la forme fonctionnelle est à choisir (linéaire, quadratique, log, exponentielle, ...)


In [23]:
cpi = df['CPI']

In [1]:
conda install -c conda-forge altair vega_datasets

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::conda-pack==0.6.0=pyhd3eb1b0_0
  - defaults/noarch::jupyterlab_server==2.8.2=pyhd3eb1b0_0
  - conda-forge/osx-64::conda==4.11.0=py39h6e9494a_0
  - defaults/noarch::cookiecutter==1.7.2=pyhd3eb1b0_0
  - defaults/osx-64::astroid==2.6.6=py39hecd8cb5_0
  - defaults/osx-64::numba==0.54.1=py39hae1ba45_0
  - defaults/osx-64::jupyter==1.0.0=py39hecd8cb5_7
  - defaults/noarch::python-lsp-black==1.0.0=pyhd3eb1b0_0
  - defaults/osx-64::gevent==21.8.0=py39h9ed2024_1
  - defaults/osx-64::zope.event==4.5.0=py39hecd8cb5_0
  - defaults/noarch::nbclient==0.5.3=pyhd3eb1b0_0
  - defaults/noarch::conda-repo-cli==1.0.4=pyhd3eb1b0_0
  - defaults/noarch::backports.functools_lru_cache==1.6.4=pyhd3eb1b0_0
  - defaults/osx-64::nbconvert==6.1.0=py39hecd8cb5_0
  - defaults/osx-64::ipykernel==6.4.1=py39hecd8cb5_1
  - defaults/noarch::nbclassic==

done


==> WARNING: A newer version of conda exists. <==
  current version: 4.11.0
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /usr/local/anaconda3

  added / updated specs:
    - altair
    - vega_datasets


The following NEW packages will be INSTALLED:

  _anaconda_depends  pkgs/main/osx-64::_anaconda_depends-2021.11-py39_0
  altair             conda-forge/noarch::altair-4.2.0-pyhd8ed1ab_1
  setuptools         conda-forge/osx-64::setuptools-62.2.0-py39h6e9494a_0
  statsmodels        conda-forge/osx-64::statsmodels-0.13.2-py39hc89836e_0
  vega_datasets      conda-forge/noarch::vega_datasets-0.9.0-pyhd3deb0d_0

The following packages will be UPDATED:

  openssl              pkgs/main::openssl-1.1.1l-h9ed2024_0 --> conda-forge::openssl-1.1.1o-hfe4f2af_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.10.26~ --> conda

##### 3. Stationnariser la série de CPI en utilisant un moyenne mobile centrée 5x5.


##### 4. Calculer inf, le taux d’inflation à partir de la variable CPI. Faire un graphique dans le temps de inf. Commentez.

###### 5. Interpréter l'autocorrélogramme et l'autocorrélogrammes partiels de inf. Quelle est la différence entre ces deux graphiques ?

##### 6. Quelle est la différence entre la stationnarité et l'ergodicité ? Pourquoi a-t-on besoin de ces deux conditions. Expliquez le terme "spurious regression".

##### 7. Faire le test Augmented Dickey Fuller pour inf en utilisant utilisant le critère AIC pour déterminer le nombre de lags à inclure. Commenter

##### 8. Proposer une modélisation AR(p) de inf, en utilisant tous les outils vus au cours.


##### 9. Estimer le modèle de la courbe de Philips qui explique le taux de chômage (Unemp) en fonction du taux d’inflation courant et une constante.

##### 10.  Tester l’autocorrélation des erreurs.

##### 11. Corriger l’autocorrélation des erreurs par la méthode vue en cours.. 

##### 12.  Tester la stabilité de la relation chômage-inflation sur deux sous-périodes de taille identique.


##### 13. Estimer la courbe de Philips en supprimant l'inflation courante des variables explicatives mais en ajoutant les délais d’ordre 1, 2, 3 et 4 de l’inflation et du chômage. Faire le test de Granger de non causalité de l’inflation sur le chômage. Donnez la p-valeur.


##### 14. Représentez graphiquement les délais distribués et commentez. Calculer l’impact à long de terme de l’inflation sur le chômage.